In [26]:
import requests
import json
import pandas as pd

In [27]:
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36 Edg/121.0.0.0",
    "Accept-Language": 'en-US,en;q=0.9',
    "Accept-Encoding": "gzip, deflate, br, zstd",
    "Referer": "https://tiki.vn/",
    "From": "",
    "af-ac-enc-dat": "",
    "x-api-source": "pc"
}

In [28]:
URL = "https://api.tiki.vn/raiden/v2/menu-config?platform=desktop"

response = requests.get(URL, headers=HEADERS)

if response.status_code == 200:
    data = response.json()
    print("API Response:", json.dumps(data, indent=4))
else:
    print("Failed to fetch data:", response.status_code)

API Response: {
    "menu_block": {
        "title": "Danh m\u1ee5c",
        "items": [
            {
                "text": "Nh\u00e0 S\u00e1ch Tiki",
                "icon_url": "https://salt.tikicdn.com/ts/category/ed/20/60/afa9b3b474bf7ad70f10dd6443211d5f.png",
                "link": "https://tiki.vn/nha-sach-tiki/c8322"
            },
            {
                "text": "Nh\u00e0 C\u1eeda - \u0110\u1eddi S\u1ed1ng",
                "icon_url": "https://salt.tikicdn.com/ts/category/f6/22/46/7e2185d2cf1bca72d5aeac385a865b2b.png",
                "link": "https://tiki.vn/nha-cua-doi-song/c1883"
            },
            {
                "text": "\u0110i\u1ec7n Tho\u1ea1i - M\u00e1y T\u00ednh B\u1ea3ng",
                "icon_url": "https://salt.tikicdn.com/ts/category/54/c0/ff/fe98a4afa2d3e5142dc8096addc4e40b.png",
                "link": "https://tiki.vn/dien-thoai-may-tinh-bang/c1789"
            },
            {
                "text": "\u0110\u1ed3 Ch\u01a1i - M\u1eb9 & B\

### GET GROUP ID

In [29]:
# Extract group data from API respone
group = data["menu_block"]["items"]
group_list = []

for category in group:
    text = category["text"]
    link = category["link"]
    # Extract group ID from the link
    group_id = link.split("/")[-1][1:]  
    group_list.append([group_id, text])

In [30]:
# Convert to DataFrame
group = pd.DataFrame(group_list, columns=["Group_ID", "Name"])
group

,Group_ID,Name
0,8322,Nhà Sách Tiki
1,1883,Nhà Cửa - Đời Sống
2,1789,Điện Thoại - Máy Tính Bảng
3,2549,Đồ Chơi - Mẹ & Bé
4,1815,Thiết Bị Số - Phụ Kiện Số
5,1882,Điện Gia Dụng
6,1520,Làm Đẹp - Sức Khỏe
7,8594,Ô Tô - Xe Máy - Xe Đạp
8,931,Thời trang nữ
9,4384,Bách Hóa Online


### GET CATEGORY ID

In [31]:
parent_category_list = []
child_category_list = []

for group_id in group["Group_ID"]:
    parent_url = f"https://tiki.vn/api/v2/categories?parent_id={group_id}"   
    parent_response = requests.get(parent_url, headers=HEADERS)
    
    if parent_response.status_code == 200:
        parent_data = parent_response.json()
        if not parent_data["data"]:
            parent_category_list.append([None, None, group_id, group[group["Group_ID"] == group_id]["Name"].item()])
        else:
            for parent_category in parent_data["data"]:
                parent_id = parent_category["id"]
                parent_name = parent_category["name"]
                
                # Get group name 
                group_name = group[group["Group_ID"] == group_id]["Name"].item()
                
                parent_category_list.append([parent_id, parent_name, group_id, group_name])
                
                # Now fetch child categories
                child_url = f"https://tiki.vn/api/v2/categories?parent_id={parent_id}" 
                child_response = requests.get(child_url, headers=HEADERS)
                
                if child_response.status_code == 200:
                    child_data = child_response.json()
                    if not child_data["data"]:
                        child_category_list.append([None, None, parent_id, parent_name, group_id, group_name])
                    else:
                        for child_category in child_data["data"]:
                            child_id = child_category["id"]
                            child_name = child_category["name"]
                            
                            child_category_list.append([child_id, child_name, parent_id, parent_name, group_id, group_name])

parent_category = pd.DataFrame(parent_category_list, columns=["Parent_ID", "Parent_Name", "Group_ID", "Group_Name"])
child_category = pd.DataFrame(child_category_list, columns=["Child_ID", "Child_Name", "Parent_ID", "Parent_Name", "Group_ID", "Group_Name"])

In [32]:
# Testing where Group_ID = 1789
child_category[child_category["Group_ID"] == "1789"]

,Child_ID,Child_Name,Parent_ID,Parent_Name,Group_ID,Group_Name
152,NaN,None,1795,Điện thoại Smartphone,1789,Điện Thoại - Máy Tính Bảng
153,NaN,None,1794,Máy tính bảng,1789,Điện Thoại - Máy Tính Bảng
154,NaN,None,28856,Máy đọc sách,1789,Điện Thoại - Máy Tính Bảng
155,NaN,None,1796,Điện thoại phổ thông,1789,Điện Thoại - Máy Tính Bảng
156,NaN,None,8061,Điện thoại bàn,1789,Điện Thoại - Máy Tính Bảng


### GET TYPE ID

In [40]:
type_info_list = []

for _, row in child_category.iterrows():
    child_id = row["Child_ID"]
    child_name = row["Child_Name"]
    parent_id = row["Parent_ID"]
    parent_name = row["Parent_Name"]
    group_id = row["Group_ID"]
    group_name = row["Group_Name"]

    if pd.isna(child_id):
        type_info_list.append([None, None, None, child_name, parent_id, parent_name, group_id, group_name])
    else:
        type_url = f"https://tiki.vn/api/v2/categories?parent_id={child_id}"
        type_response = requests.get(type_url, headers=HEADERS)

        if type_response.status_code == 200:
            type_data = type_response.json()
            if type_data["data"]:
                for type_item in type_data["data"]:
                    type_id = type_item.get("id")
                    type_name = type_item.get("name")
                    type_info_list.append([type_id, type_name, child_id, child_name, parent_id, parent_name, group_id, group_name])
            else:
                type_info_list.append([None, None, child_id, child_name, parent_id, parent_name, group_id, group_name])

type_info = pd.DataFrame(type_info_list, columns=["Type_ID", "Type_Name", "Child_ID", "Child_Name", "Parent_ID", "Parent_Name", "Group_ID", "Group_Name"])
print(type_info)

      Type_ID                 Type_Name  Child_ID         Child_Name  \
0       625.0              Architecture     623.0  Art & Photography   
1      5770.0            Graphic Design     623.0  Art & Photography   
2       626.0         Religion, Culture     623.0  Art & Photography   
3      9423.0               Photography     623.0  Art & Photography   
4       112.0  Decorative Arts & Design     623.0  Art & Photography   
...       ...                       ...       ...                ...   
2213      NaN                      None       NaN               None   
2214      NaN                      None       NaN               None   
2215      NaN                      None       NaN               None   
2216      NaN                      None   49652.0       Ví nam ngang   
2217      NaN                      None   49654.0        Ví nam đứng   

      Parent_ID               Parent_Name Group_ID          Group_Name  
0           320             English Books     8322       Nhà S

In [ ]:
# import pandas as pd

# type_df = pd.DataFrame(columns=['Type_ID', 
#                                 'Type_Name', 
#                                 'Child_ID', 
#                                 'Child_Name', 
#                                 'Parent_ID', 
#                                 'Parent_Name', 
#                                 'Group_ID', 
#                                 'Group_Name'])

# for index, row in child_category.iterrows():
#     child_id = row['Child_ID']
#     if pd.isnull(child_id) or child_id == '':
#         new_row = {'Type_ID': None, 
#                    'Type_Name': None, 
#                    'Child_ID': row['Child_ID'], 
#                    'Child_Name': row['Child_Name'], 
#                    'Parent_ID': row['Parent_ID'], 
#                    'Parent_Name': row['Parent_Name'], 
#                    'Group_ID': row['Group_ID'], 
#                    'Group_Name': row['Group_Name']}
#         type_df = pd.concat([type_df, pd.DataFrame([new_row])], ignore_index=True)
#     else:
#         type_url = f"https://tiki.vn/api/v2/categories?parent_id={child_id}"
#         type_response = requests.get(type_url, headers=HEADERS)
#         type_data = type_response.json()
#         if isinstance(type_data, dict):  # Check if type_data is a dictionary
#             for type_item in type_data.get("data", []):  # Handle if "data" key is missing
#                 type_id = type_item.get("id")
#                 type_name = type_item.get("name")
#                 new_row = {'Type_ID': type_id, 
#                            'Type_Name': type_name, 
#                            'Child_ID': row['Child_ID'], 
#                            'Child_Name': row['Child_Name'], 
#                            'Parent_ID': row['Parent_ID'], 
#                            'Parent_Name': row['Parent_Name'], 
#                            'Group_ID': row['Group_ID'], 
#                            'Group_Name': row['Group_Name']}
#                 type_df = pd.concat([type_df, pd.DataFrame([new_row])], ignore_index=True)

In [41]:
type_info

,Type_ID,Type_Name,Child_ID,Child_Name,Parent_ID,Parent_Name,Group_ID,Group_Name
0,625.0,Architecture,623.0,Art & Photography,320,English Books,8322,Nhà Sách Tiki
1,5770.0,Graphic Design,623.0,Art & Photography,320,English Books,8322,Nhà Sách Tiki
2,626.0,"Religion, Culture",623.0,Art & Photography,320,English Books,8322,Nhà Sách Tiki
3,9423.0,Photography,623.0,Art & Photography,320,English Books,8322,Nhà Sách Tiki
4,112.0,Decorative Arts & Design,623.0,Art & Photography,320,English Books,8322,Nhà Sách Tiki
...,...,...,...,...,...,...,...,...
2213,NaN,None,NaN,None,5337,Túi xách công sở nam,27616,Túi thời trang nam
2214,NaN,None,NaN,None,958,Túi đeo chéo nam,27616,Túi thời trang nam
2215,NaN,None,NaN,None,49650,"Túi bao tử, túi đeo bụng",27616,Túi thời trang nam
2216,NaN,None,49652.0,Ví nam ngang,959,Ví nam,27616,Túi thời trang nam


In [42]:
# Testing where Type ID is null
type_info[type_info["Type_ID"].isnull() & type_info["Child_ID"].isnull()]

,Type_ID,Type_Name,Child_ID,Child_Name,Parent_ID,Parent_Name,Group_ID,Group_Name
491,NaN,None,NaN,None,1795,Điện thoại Smartphone,1789,Điện Thoại - Máy Tính Bảng
492,NaN,None,NaN,None,1794,Máy tính bảng,1789,Điện Thoại - Máy Tính Bảng
493,NaN,None,NaN,None,28856,Máy đọc sách,1789,Điện Thoại - Máy Tính Bảng
494,NaN,None,NaN,None,1796,Điện thoại phổ thông,1789,Điện Thoại - Máy Tính Bảng
495,NaN,None,NaN,None,8061,Điện thoại bàn,1789,Điện Thoại - Máy Tính Bảng
515,NaN,None,NaN,None,6568,Thực phẩm ăn dặm,2549,Đồ Chơi - Mẹ & Bé
516,NaN,None,NaN,None,10416,Dinh dưỡng cho mẹ,2549,Đồ Chơi - Mẹ & Bé
517,NaN,None,NaN,None,10418,Dinh dưỡng cho người lớn,2549,Đồ Chơi - Mẹ & Bé
1263,NaN,None,NaN,None,49384,Thời trang trung niên,931,Thời trang nữ
1883,NaN,None,NaN,None,3862,Máy giặt,4221,Điện Tử - Điện Lạnh


In [43]:
# Testing where Group_ID = 1789
type_info[type_info["Group_ID"] == "1789"]

,Type_ID,Type_Name,Child_ID,Child_Name,Parent_ID,Parent_Name,Group_ID,Group_Name
491,NaN,None,NaN,None,1795,Điện thoại Smartphone,1789,Điện Thoại - Máy Tính Bảng
492,NaN,None,NaN,None,1794,Máy tính bảng,1789,Điện Thoại - Máy Tính Bảng
493,NaN,None,NaN,None,28856,Máy đọc sách,1789,Điện Thoại - Máy Tính Bảng
494,NaN,None,NaN,None,1796,Điện thoại phổ thông,1789,Điện Thoại - Máy Tính Bảng
495,NaN,None,NaN,None,8061,Điện thoại bàn,1789,Điện Thoại - Máy Tính Bảng


In [44]:
# # Store to test (encoding utf-8-sig to avoid BOM)
type_info.to_csv("tiki_type_automatic1.csv", index=False, encoding="utf-8-sig")